# 預處理流程

清emoji、@@username、url -> 存成clean_text

In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('comments/face2face_comments_merged.csv', encoding='utf-8')
print(f"一共有 {len(df)} 筆留言")

一共有 105186 筆留言


In [4]:
# 取不重複留言者姓名

def name_list(data):
    data['author_name'] = data['author_name'].apply(lambda x: str(x).replace('@', ''))

    name_list = []
    for i, name in enumerate(data['author_name']):
        name_str = str(name)
        if name_str not in name_list:
            name_list.append(name_str)

    return name_list

names = name_list(df)
print(">>> name_list:", names)  
print(f"一共有 {len(names)} 個不重複留言者")

>>> name_list: ['vintw', '蔡奇順-c8i', 'timwow2655', 'sandy102211', 'Sun-rp2ze', '莊榮峰-t4s', 'Lily-vr7hm', 'wokastal', '李古汀', 'magicityjack3018', 'hungchiwen', 'Mr.weamonliu', 'weizon4975', 'k780701', 'alexhsiao1677', '梁芷甄', 'JarryLu', 'clarenceyu279', '小山丘-c1c', 'mikemike291', 'dandansfu', '溫正大', 'loreanon', 'chungweijen', '眞野ミオン', 'Koala-tw', 'chenctc', 'dovywon2790', 'universeforcecolumn', 'Pan0412', '葉惠珠-i3s', 'zj2oclijh8iloe2e', '姜宇-x9i', 'chuws1265', 'person198319', 'tangkiahming4663', 'f778899', 'lu5382', 'Mimiya2733', 'ccy1330', 'melissahuang9545', 'Yessir88888', 'ronaldtsai6774', 'utaemi7777', 'deIiahsiang', 'hyde0313', '東海林藥', 'greatlazycat', '林泰成-h2b', 'flyeric1125', 'shih.8688', 'angelajen6358', 'qqdiploxxxxxx', '羊-c3g', 'fakerich', 'tangerine_ovob', 'RICHARDSON513', 'Sheets-o8q', 'renejyo', 'xinphy1607', 'adidas71409', 'sinloverin1', 'silverapin5167', 'WindowXDD', 'jenyu1199', 'Oscar-kv6qm', '謝孟諭-n4n', 'SN-ib5pm', '1450lwy3q', '黃詩喬-r3p', 'Conan-1984', 'cacklecollection', '楊宗翰-

In [ ]:
def clean_text(comment_data, name_list):
    comment_data['cleaned_text'] = None

    for i, c in enumerate(comment_data['comment_text']):
        print(f"{i+1} / {len(comment_data)}")

        # 移除 HTML 標籤、特殊符號、author_name
        c1 = re.sub(r'<br>|<a href=".*?">.*?</a>|<\/?b>', ' ', str(c))
        c2 = re.sub(r'[^\w\s,]', ' ', c1)
        c3 = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fa5]', ' ', c2)
        
        c_cleaned = c3
        for name in name_list:
            c_cleaned = re.sub(r'\b' + re.escape(name) + r'\b', '', c_cleaned)

        comment_data.at[i, 'cleaned_text'] = c_cleaned
        # print(f"{c} -> {c_cleaned}")

    original = comment_data.groupby('video_title').size()
    print(">>> 原始留言數量：", original)

    # 去除空白留言
    df_cleaned = comment_data[comment_data['cleaned_text'].str.strip() != '']
    after_drop = df_cleaned.groupby('video_title').size()
    print(">>> 去除空白留言數量：", after_drop)

    # 對齊 index，沒有的補 0
    after_drop = after_drop.reindex(original.index, fill_value=0)
    drop_count = (original - after_drop).reset_index()
    drop_count.columns = ['video_title', 'count']

    # save diff and dropped data to csv
    df_cleaned.to_csv('comments/face2face_cleaned.csv', index=False)
    drop_count.to_csv('comments/face2face_drop_count.csv', index=False)

    return df_cleaned

cleaned_df = clean_text(df, names)
print(len(cleaned_df))

1 / 105186
2 / 105186
3 / 105186
4 / 105186
5 / 105186
6 / 105186
7 / 105186
8 / 105186
9 / 105186
10 / 105186
11 / 105186
12 / 105186
13 / 105186
14 / 105186
15 / 105186
16 / 105186
17 / 105186
18 / 105186
19 / 105186
20 / 105186
21 / 105186
22 / 105186
23 / 105186
24 / 105186
25 / 105186
26 / 105186
27 / 105186
28 / 105186
29 / 105186
30 / 105186
31 / 105186
32 / 105186
33 / 105186
34 / 105186
35 / 105186
36 / 105186
37 / 105186
38 / 105186
39 / 105186
40 / 105186
41 / 105186
42 / 105186
43 / 105186
44 / 105186
45 / 105186
46 / 105186
47 / 105186
48 / 105186
49 / 105186
50 / 105186
51 / 105186
52 / 105186
53 / 105186
54 / 105186
55 / 105186
56 / 105186
57 / 105186
58 / 105186
59 / 105186
60 / 105186
61 / 105186
62 / 105186
63 / 105186
64 / 105186
65 / 105186
66 / 105186
67 / 105186
68 / 105186
69 / 105186
70 / 105186
71 / 105186
72 / 105186
73 / 105186
74 / 105186
75 / 105186
76 / 105186
77 / 105186
78 / 105186
79 / 105186
80 / 105186
81 / 105186
82 / 105186
83 / 105186
84 / 105186
8

# 計算互動

- 技術性互動：top_comment/reply/single
- 實質討論：reply/@@username

In [8]:
df = pd.read_csv('comments/face2face_cleaned.csv')

In [14]:

def tag_interact(df, name_list):
    """
    top_interaction: 留言串的第一則
    single: 沒有任何互動的單一留言
    reply: comment_type == "reply" or @@usename in comment_text
    """
    video_id_map = {title: idx + 1 for idx, title in enumerate(df['video_title'].unique())}
    df['video_id'] = df['video_title'].map(video_id_map)

    # 建立結果列表
    discussion_results = []
    
    # 針對每部影片獨立處理
    for vid in df['video_id'].unique():
        mask = df['video_id'] == vid  # mask是用來抓出特定影片資料
        print(f"=== 處理影片 {vid} ===")
        sub_df = df[mask].reset_index()

        for i in range(len(sub_df)):
            current_row = sub_df.iloc[i].copy()
            current_type = current_row['comment_type']
            next_type = sub_df.iloc[i + 1]['comment_type'] if i + 1 < len(sub_df) else None

            if current_type == 'top_comment' and next_type == 'reply':
                current_row['interaction_type'] = "top_interaction"
            
            elif current_type == 'top_comment' and next_type == 'top_comment':
                for name in name_list:
                    if f"@@{str(name)}" in current_row['comment_text']:
                        current_row['interaction_type'] = "reply"
                    else:
                        current_row['interaction_type'] = "single"
            
            elif current_type == 'reply':
                current_row['interaction_type'] = "reply"

            discussion_results.append(current_row)
    
    # 將結果轉換為 DataFrame
    result_df = pd.DataFrame(discussion_results)
    
    # 儲存結果到 CSV
    result_df.to_csv('comments/face2face_discussion_group.csv', index=False)
    
    return result_df

interaction_tag = tag_interact(df, names)

=== 處理影片 1 ===
=== 處理影片 2 ===
=== 處理影片 3 ===
=== 處理影片 4 ===
=== 處理影片 5 ===
=== 處理影片 6 ===
=== 處理影片 7 ===
=== 處理影片 8 ===
=== 處理影片 9 ===
=== 處理影片 10 ===
=== 處理影片 11 ===
=== 處理影片 12 ===
=== 處理影片 13 ===
=== 處理影片 14 ===
=== 處理影片 15 ===
=== 處理影片 16 ===
=== 處理影片 17 ===
=== 處理影片 18 ===
=== 處理影片 19 ===
=== 處理影片 20 ===
=== 處理影片 21 ===


In [16]:
# count interaction type
def count_dc_group(df: pd.DataFrame) -> pd.DataFrame:
    result = []

    for vid in df['video_id'].unique():
        sub_df = df[df['video_id'] == vid]
        video_title = sub_df['video_title'].iloc[0]

        # 實質互動留言數：有參與討論串的留言
        interactive_comments = sub_df[sub_df['interaction_type'] != 'single'].shape[0]

        # 總留言數
        total_comments = sub_df.shape[0]

        # 實質互動比例
        interaction_ratio = interactive_comments / total_comments

        result.append({
            'video_id': vid,
            'video_title': video_title,
            'total_comments': total_comments,
            'interactive_comments': interactive_comments,
            'interaction_ratio': round(interaction_ratio, 4)
        })

    result_df = pd.DataFrame(result)
    result_df.to_csv('comments/face2face_discussion_counts.csv', index=False)

    return result_df

interaction_counts = count_dc_group(interaction_tag)

# 跑bertopic